In [ ]:
import requests

# 세노바메이트의 CID (사용자님이 찾으신 번호)
cid = "11962412"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/JSON"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # PubChem의 전체 JSON 구조는 매우 복잡하므로, 
    # 'Section' 내부에서 'LogP'라는 이름을 가진 데이터를 찾아야 합니다.
    
    # 팁: 실제 API 응답에서 'Section' 데이터를 파싱하는 로직이 필요합니다.
    # 아래는 간단한 출력 확인용입니다.
    print(f"--- CID {cid} (세노바메이트) 데이터 로드 성공 ---")
    print("웹페이지에서 보신 0.456은 '실험치' 섹션에 저장되어 있습니다.")
else:
    print("데이터를 불러오지 못했습니다.")

--- CID 11962412 (세노바메이트) 데이터 로드 성공 ---
웹페이지에서 보신 0.456은 '실험치' 섹션에 저장되어 있습니다.


In [ ]:
import pandas as pd

def transform_pubchem_to_df(raw_api_data):
    all_compounds = []

    # 1. 데이터 구조 진입 (PC_Compounds 리스트 순회)
    # data['PC_Compounds'] 내의 각 화합물 정보를 가져옵니다.
    for compound_info in raw_api_data.get('PC_Compounds', []):
        row = {}
        
        props = compound_info.get('props', [])
        for p in props:
            label = p['urn'].get('label', '')
            name = p['urn'].get('name', '')
            
            # 컬럼명 생성 (예: Count_Hydrogen Bond Donor)
            col_name = f"{label}_{name}".strip('_')
            
            # 값 추출 (fval, ival, sval 중 있는 것을 선택)
            val_dict = p.get('value', {})
            val = None
            if 'fval' in val_dict: val = val_dict['fval']
            elif 'ival' in val_dict: val = val_dict['ival']
            elif 'sval' in val_dict: val = val_dict['sval']
            
            row[col_name] = val
        
        # 3. 추가 정보 (CID 등)가 있다면 같이 저장
        if 'id' in compound_info:
            row['CID'] = compound_info['id'].get('id', {}).get('cid')

        all_compounds.append(row)

    # 4. DataFrame 생성
    df = pd.DataFrame(all_compounds)
    
    # 5. 수치 데이터 형식 변환 (문자열로 들어온 숫자를 float으로)
    numeric_cols = ['Molecular Weight', 'Log P_XLogP3-AA', 'Complexity', 'Topological_Polar Surface Area']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
    return df

# 사용 예시:
df = transform_pubchem_to_df(data)
print(df)
df.to_csv('pubchem_features.csv', index=False)

   Compound_Canonicalized  Compound Complexity  Count_Hydrogen Bond Acceptor  \
0                       1                  293                             5   

   Count_Hydrogen Bond Donor  Count_Rotatable Bond  \
0                          1                     5   

  Fingerprint_SubStructure Keys  \
0                          None   

                                  IUPAC Name_Allowed  \
0  [(1R)-1-(2-chlorophenyl)-2-(tetrazol-2-yl)ethy...   

                           IUPAC Name_CAS-like Style  \
0  carbamic acid [(1R)-1-(2-chlorophenyl)-2-(2-te...   

                                   IUPAC Name_Markup  \
0  [(1<I>R</I>)-1-(2-chlorophenyl)-2-(tetrazol-2-...   

                                IUPAC Name_Preferred  ...  \
0  [(1R)-1-(2-chlorophenyl)-2-(tetrazol-2-yl)ethy...  ...   

             InChIKey_Standard Log P_XLogP3-AA   Mass_Exact Molecular Formula  \
0  GFHAXPJGXSQLPT-VIFPVBQESA-N             1.5  267.0523023      C10H10ClN5O2   

   Molecular Weight               

In [26]:
df['Log P_XLogP3-AA']
df['Molecular Weight']

0    267.67
Name: Molecular Weight, dtype: float64